## Dataload

In [25]:
import requests
import pandas as pd

# API 요청을 자동화하여 데이터 가져오기
def get_data(start, end):
    url = f'http://openapi.seoul.go.kr:8088/5648446357726b6437355358536443/json/GarakGradePrice/{start}/{end}/'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data['GarakGradePrice']['row'])

df = pd.DataFrame()

# 1000개씩 데이터 가져와서 합치기
for i in range(1, 3000, 1000):
    data = get_data(i, i + 999)
    df = pd.concat([df, data])

# 인덱스 재설정
df.reset_index(drop=True, inplace=True)

# 결과 출력
df = df[~((df['MAXPRICE']==0) & (df['MAXPRICE']==0) & (df['MAXPRICE']==0))]# 최고가 0인것들 제외(이상치)
df['kg당 평균 가격'] = (df['AVGPRICE'] / df['UNITQTY']).astype('int')
df


,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격
0,(냉)갈치,상,10.0,kg상자,350000.0,26667.0,90770.0,20240314,9077
1,(냉)갈치,중,10.0,kg상자,26667.0,25000.0,26276.0,20240314,2627
2,(냉)고등어,상,10.0,kg상자,80000.0,27000.0,40154.0,20240314,4015
4,(냉)고등어 수입,상,10.0,kg상자,52000.0,32750.0,35325.0,20240314,3532
8,(선)갈치,상,5.0,kg상자,190000.0,180000.0,185469.0,20240314,37093
...,...,...,...,...,...,...,...,...,...
2011,활 방어(자연),중,1.0,kg,9000.0,4000.0,6328.0,20240314,6328
2012,활 방어(자연),소,1.0,kg,4000.0,3000.0,3590.0,20240314,3590
2017,활 전복(양식),상,1.0,kg,31000.0,21000.0,25648.0,20240314,25648
2018,활 전복(양식),중,1.0,kg,21000.0,13000.0,18123.0,20240314,18123


In [26]:
def category(item):
  su_list = ["갈치", "고등어", "멸치", "조개", "게", "새우", "오징어", "낙지", "문어", "해삼", "고래", "바다", "물고기" , '활' , '가자미' , '명태' , '어','홍합','소라','굴',
             "근대", "깐 바지락", "냉태", "대구", "달래", "매생이", "머위대", "삼치",'조기' ,"부세", "주꾸미"]

  ve_list = ["가지", "감자", "양파", "배추", "무", "당근", "토마토", "오이", "버섯", "고추", "피망", "깻잎", "시금치", "열무", "콜리플라워", "브로콜리", "아스파라거스",
              "양배추", "샐러리", "케일", "깻잎", "깻잎순", "쑥갓" ,'고구마','고수','로메인','호박','미나리','콜라비','미더덕','아귀','칼리',"김", 
              "깐마늘", "냉이", "노랑 파프리카", "당귀잎", "대파", "동죽", "레드 치커리","생표고", "나물" , '파'
                 "로케트 루꼴라", "매생이", "머위대", "바실",  "부추", "블루베리", "비타민","비트", "빨강 파프리카", "상추", "생표고", "셀러리", "아욱","양송이", "부추",
                 "오만둥이", "적근대", "적상추", "쪽파", "쫑상추",  "참두릅", "청경채",
                 "취나물", "치커리",'루꼴라']
  fru_list = ['귤','배','사과','샤인머스캣','파인애플','수박','딸기','멜론','바나나',"단감", "만감", "참외", "오렌지"]

  # 품목명에 키워드 포함 여부 확인
  for keyword in su_list:
    if keyword in item:
      return "수산물"
  for keyword in ve_list:
    if keyword in item:
      return "야채"
  for keyword in fru_list:
    if keyword in item:
      return "과일"
  return "기타"

df['category'] = df['PUMNAME'].apply(category)
df

,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격,category
0,(냉)갈치,상,10.0,kg상자,350000.0,26667.0,90770.0,20240314,9077,수산물
1,(냉)갈치,중,10.0,kg상자,26667.0,25000.0,26276.0,20240314,2627,수산물
2,(냉)고등어,상,10.0,kg상자,80000.0,27000.0,40154.0,20240314,4015,수산물
4,(냉)고등어 수입,상,10.0,kg상자,52000.0,32750.0,35325.0,20240314,3532,수산물
8,(선)갈치,상,5.0,kg상자,190000.0,180000.0,185469.0,20240314,37093,수산물
...,...,...,...,...,...,...,...,...,...,...
2011,활 방어(자연),중,1.0,kg,9000.0,4000.0,6328.0,20240314,6328,수산물
2012,활 방어(자연),소,1.0,kg,4000.0,3000.0,3590.0,20240314,3590,수산물
2017,활 전복(양식),상,1.0,kg,31000.0,21000.0,25648.0,20240314,25648,수산물
2018,활 전복(양식),중,1.0,kg,21000.0,13000.0,18123.0,20240314,18123,수산물


In [27]:
df_test = df[['PUMNAME','UNITNAME','category']]
lst = list(df[df['category']=='기타']['PUMNAME'].values)
lst

['그린키위 국산',
 '그린키위 국산',
 '그린키위 국산',
 '그린키위 국산',
 '머위잎',
 '머위잎',
 '머위잎',
 '파슬리',
 '파슬리',
 '파슬리']

In [28]:
X = df.drop('category',axis=1)
Y = df[['category']]
X.head()

,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격
0,(냉)갈치,상,10.0,kg상자,350000.0,26667.0,90770.0,20240314,9077
1,(냉)갈치,중,10.0,kg상자,26667.0,25000.0,26276.0,20240314,2627
2,(냉)고등어,상,10.0,kg상자,80000.0,27000.0,40154.0,20240314,4015
4,(냉)고등어 수입,상,10.0,kg상자,52000.0,32750.0,35325.0,20240314,3532
8,(선)갈치,상,5.0,kg상자,190000.0,180000.0,185469.0,20240314,37093


In [29]:
x_train = X[~X['PUMNAME'].isin(lst)]
y_train =  Y[~(Y['category']=='기타')]
test = X[X['PUMNAME'].isin(lst)]

In [30]:
x_train.shape , y_train.shape , test.shape 

((773, 9), (773, 1), (10, 9))

In [31]:
from sklearn.preprocessing import LabelEncoder

combined_data = pd.concat([x_train, test], axis=0) 

'''
아래에 붙이는 이유는 필요한 데이터만 추출해서 분리시킨후 아래로 배치 시키려고
따로 train,test로 각각 라벨 인코딩을 적용하면 train에 없는 데이터가 test에 있다고 인코딩이 진행되지 않는다.
(테스트 데이터에 학습 데이터에 없는 카테고리가 포함오류)
'''
cat_col = combined_data.select_dtypes(include='object').columns

le = LabelEncoder()
for column in cat_col:
    combined_data[column] = le.fit_transform(combined_data[column])

x_train_encoded = combined_data[:len(x_train)]
test_encoded = combined_data[len(x_train):]

In [32]:
test

,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격
120,그린키위 국산,특,10.0,kg상자,54000.0,54000.0,54000.0,20240314,5400
121,그린키위 국산,상,10.0,kg상자,54000.0,53000.0,53480.0,20240314,5348
122,그린키위 국산,중,10.0,kg상자,53000.0,48000.0,51378.0,20240314,5137
123,그린키위 국산,하,10.0,kg상자,48000.0,22000.0,36125.0,20240314,3612
638,머위잎,상,4.0,kg상자,44000.0,30000.0,39416.0,20240314,9854
639,머위잎,중,4.0,kg상자,30000.0,27500.0,28680.0,20240314,7170
640,머위잎,하,4.0,kg상자,27500.0,23000.0,24715.0,20240314,6178
982,파슬리,상,4.0,kg상자,13000.0,12500.0,12667.0,20240314,3166
983,파슬리,중,4.0,kg상자,12500.0,11000.0,11995.0,20240314,2998
984,파슬리,하,4.0,kg상자,11000.0,8000.0,9600.0,20240314,2400


In [33]:
def category(item):
  fru_list = ["키위"]
  ve_list = ['머위잎','파슬리']

  # 품목명에 키워드 포함 여부 확인
  for keyword in ve_list:
    if keyword in item:
      return "야채"
  for keyword in fru_list:
    if keyword in item:
      return "과일"
    
test_copy = test.copy()
test_copy.loc[:, 'category'] = test['PUMNAME'].apply(category)
y_test = test_copy['category']

In [34]:
Y = df['category']

In [35]:
Y

0       수산물
1       수산물
2       수산물
4       수산물
8       수산물
       ... 
2011    수산물
2012    수산물
2017    수산물
2018    수산물
2021    수산물
Name: category, Length: 783, dtype: object

In [36]:
x_train_encoded.shape , y_train.shape , test_encoded.shape , y_test.shape

((773, 9), (773, 1), (10, 9), (10,))

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
rf_model = RandomForestClassifier()
g_model = GradientBoostingClassifier()
g_model.fit(x_train_encoded, y_train)
pred = g_model.predict(test_encoded)
pred

c:\Users\User\document\공부\project\sijang_project\myvenv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['야채', '야채', '야채', '야채', '과일', '야채', '야채', '야채', '야채', '야채'],
      dtype=object)

In [38]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

Accuracy: 0.5


In [39]:
test['예측'] = pred # 이건 그냥 확인용
test.rename(columns={'category': '실제'}, inplace=True)
test

C:\Users\User\AppData\Local\Temp\ipykernel_13956\7291874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['예측'] = pred # 이건 그냥 확인용
C:\Users\User\AppData\Local\Temp\ipykernel_13956\7291874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.rename(columns={'category': '실제'}, inplace=True)


,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격,예측
120,그린키위 국산,특,10.0,kg상자,54000.0,54000.0,54000.0,20240314,5400,야채
121,그린키위 국산,상,10.0,kg상자,54000.0,53000.0,53480.0,20240314,5348,야채
122,그린키위 국산,중,10.0,kg상자,53000.0,48000.0,51378.0,20240314,5137,야채
123,그린키위 국산,하,10.0,kg상자,48000.0,22000.0,36125.0,20240314,3612,야채
638,머위잎,상,4.0,kg상자,44000.0,30000.0,39416.0,20240314,9854,과일
639,머위잎,중,4.0,kg상자,30000.0,27500.0,28680.0,20240314,7170,야채
640,머위잎,하,4.0,kg상자,27500.0,23000.0,24715.0,20240314,6178,야채
982,파슬리,상,4.0,kg상자,13000.0,12500.0,12667.0,20240314,3166,야채
983,파슬리,중,4.0,kg상자,12500.0,11000.0,11995.0,20240314,2998,야채
984,파슬리,하,4.0,kg상자,11000.0,8000.0,9600.0,20240314,2400,야채


In [41]:
x_train = df[['PUMNAME','UNITNAME']]
y_train = df[['category']]

In [42]:
x_train.shape , y_train.shape , test.shape

((783, 2), (783, 1), (10, 10))

In [43]:
x_train = pd.get_dummies(x_train)
test = pd.get_dummies(test)
X_train.shape ,  test.shape

NameError: name 'X_train' is not defined

In [ ]:
test

,PUMNAME_다시마,PUMNAME_머위잎,PUMNAME_파슬리,UNITNAME_kg상자
333,True,False,False,True
572,False,True,False,True
573,False,True,False,True
574,False,True,False,True
1857,False,False,True,True
1858,False,False,True,True
1859,False,False,True,True


In [ ]:
test

,PUMNAME_다시마,PUMNAME_머위잎,PUMNAME_파슬리,UNITNAME_kg상자
333,True,False,False,True
572,False,True,False,True
573,False,True,False,True
574,False,True,False,True
1857,False,False,True,True
1858,False,False,True,True
1859,False,False,True,True


In [ ]:
test = test.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
test.shape

(7, 4)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)
pred = model.predict(test)

c:\Users\User\document\공부\project\sijang_project\myvenv\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\User\document\공부\project\sijang_project\myvenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 4 features, but RandomForestClassifier is expecting 215 features as input.